<a href="https://colab.research.google.com/github/DrFrank25/Syndecan_4-Ag73/blob/main/gmx_MMPBSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Binding Free Energy Calculation (MM-PBSA + Entropy)**

In [ ]:
# ==========================================
# 🚀 Binding Free Energy Calculation (MM-PBSA + Entropy) in Colab
# ==========================================
# -- Import Libraris ----
import os
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
# ===============================
# 1) Install GROMACS + gmx_MMPBSA
# ===============================
!mamba install -y -c conda-forge gromacs gmx-mmpbsa


In [ ]:
# ===============================
# 2) Upload input files
# ===============================
print("📂 Please upload files for 3 replicates:")
print("   - topol1.tpr, traj1.xtc, index1.ndx, mmpbsa1.in")
print("   - topol2.tpr, traj2.xtc, index2.ndx, mmpbsa2.in")
print("   - topol3.tpr, traj3.xtc, index3.ndx, mmpbsa3.in")
uploaded = files.upload()

In [ ]:
# ===============================
# 3) Function to run MM-PBSA + Entropy
# ===============================
def run_mmpbsa_entropy(rep_id, tpr, xtc, ndx, inp):
    out_dir = f"MMPBSA_rep{rep_id}"
    os.makedirs(out_dir, exist_ok=True)

    # Run MM-PBSA with entropy calculation enabled in input (.in)
    cmd = f"g_mmpbsa -f {xtc} -s {tpr} -n {ndx} -i {inp} -o {out_dir}/FINAL_RESULTS_MMPBSA.dat -eo {out_dir}/entropy_QHA.dat -nogui"
    subprocess.run(cmd, shell=True, check=True)

    results = {}
    # Parse enthalpy and solvation contributions
    with open(f"{out_dir}/FINAL_RESULTS_MMPBSA.dat") as f:
        for line in f:
            if line.strip() and not line.startswith("#"):
                parts = line.split()
                if len(parts) >= 2:
                    try:
                        results[parts[0]] = float(parts[1])
                    except:
                        pass

    # Parse entropy term (-TΔS) from QHA
    with open(f"{out_dir}/entropy_QHA.dat") as f:
        for line in f:
            if "TΔS" in line or "TS" in line:
                parts = line.split()
                try:
                    results["-TΔS"] = float(parts[-1])  # usually last column
                except:
                    pass

    # Compute ΔH and ΔG_total
    results["ΔH"] = results.get("DeltaE_vdw",0) + results.get("DeltaE_ele",0) + results.get("DeltaG_pol",0) + results.get("DeltaG_np",0)
    results["ΔG_total"] = results["ΔH"] + results.get("-TΔS", 0)

    return results

In [ ]:
# ===============================
# 4) Run MM-PBSA for 3 replicates
# ===============================
replicas = {}
replicas["Replicate_1"] = run_mmpbsa_entropy(1, "topol1.tpr", "traj1.xtc", "index1.ndx", "mmpbsa1.in")
replicas["Replicate_2"] = run_mmpbsa_entropy(2, "topol2.tpr", "traj2.xtc", "index2.ndx", "mmpbsa2.in")
replicas["Replicate_3"] = run_mmpbsa_entropy(3, "topol3.tpr", "traj3.xtc", "index3.ndx", "mmpbsa3.in")


In [ ]:
# ===============================
# 5) Build results DataFrame
# ===============================
df = pd.DataFrame(replicas).T
df = df[["DeltaE_vdw", "DeltaE_ele", "DeltaG_pol", "DeltaG_np", "ΔH", "-TΔS", "ΔG_total"]]

# Add statistics (mean ± std)
df.loc["Mean"] = df.mean()
df.loc["Std"] = df.std()

df.to_csv("Binding_Energy_3Replicates_withEntropy.csv")
print("✅ Results saved to Binding_Energy_3Replicates_withEntropy.csv")

display(df)


In [ ]:
# ===============================
# 6) Visualization - Energy Contributions
# ===============================
plt.figure(figsize=(12,6))

df_plot = df.drop(["Mean","Std"])  # only replicates
df_plot.plot(kind="bar", figsize=(12,6), edgecolor="black")

plt.ylabel("Energy (kJ/mol)")
plt.title("Binding Free Energy Decomposition (with Entropy) - 3 Replicates")
plt.xticks(rotation=0)
plt.legend(title="Energy term")
plt.tight_layout()
plt.savefig("Energy_Decomposition_3Replicates_withEntropy.png", dpi=300)
plt.show()

print("📊 Plot saved as Energy_Decomposition_3Replicates_withEntropy.png")